In [ ]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'
import networkx
from typing import List, Tuple, Optional, Union
import matplotlib.pyplot as plt
import numpy
from qiskit.providers.aer import StatevectorSimulator
from qiskit.providers.aer import QasmSimulator
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.algorithms.minimum_eigen_solvers import MinimumEigensolverResult
from qiskit.aqua.components.optimizers import SPSA, COBYLA
from qiskit.aqua import QuantumInstance
from qiskit.circuit.library import TwoLocal
import matplotlib
from qiskit.result import Counts
from qiskit.optimization.applications.ising import max_cut
from qiskit import aqua
from qiskit import IBMQ
from qiskit.providers.aer.noise import NoiseModel
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit.optimization.applications.ising.common import sample_most_likely, random_graph
from qiskit.aqua.algorithms import VQE

# Setup

## Helper Routines

In [ ]:
def generate_binary_list(index: int) -> List[int]:
    # generate solution candidates (lists of 0's and 1's):
    # 1. bin() converts to binary string
    # 2. [:2] removes the '0b' prefix
    # 3. .zfill(N) prepends 0s until a length of N has been achieved
    return [int(digit) for digit in bin(combination)[2:].zfill(N)]

def draw_graph(colors: List[str]):
    fig, ax = plt.subplots(1, 1)
    networkx.draw_networkx(graph, node_color=colors, ax=ax, pos=positions)
    networkx.draw_networkx_edge_labels(
        graph, positions, edge_labels=networkx.get_edge_attributes(graph, "weight")
    )

def plot_convergence(means: numpy.ndarray, stddevs: Optional[numpy.ndarray] = None) -> Tuple[matplotlib.figure.Figure, matplotlib.axes.Axes]:
    fig, ax = plt.subplots(1, 1)
    iterations = numpy.arange(len(means))
    
    if stddevs is not None:
        ax.fill_between(iterations, means - stddevs, means + stddevs, alpha=0.5)
        
    ax.plot(iterations, means)
    ax.axhline(energy_diag, color="C1")
    ax.grid(True)
    ax.set_xlabel(r"$iteration$")
    ax.set_ylabel(r"$\langle H_{\mathrm{MC}} \rangle$")
    return fig, ax

def plot_parameter_convergence(parameters: numpy.ndarray) -> Tuple[matplotlib.figure.Figure, matplotlib.axes.Axes]:
    num_parameters = parameters.shape[1]
    iterations = numpy.arange(parameters.shape[0])
    
    fig, ax = plt.subplots(1, 1)
    
    for j in range(num_parameters):
        ax.plot(iterations, parameters[:, j])
    
    ax.grid(True)
    ax.set_xlabel(r"$iteration$")
    ax.set_ylabel(r"$\theta_j$")
    return fig, ax

def plot_eigenstate(eigenstate: Union[numpy.ndarray, Counts]) -> Tuple[matplotlib.figure.Figure, matplotlib.axes.Axes]:
    fig, ax = plt.subplots(1, 1)
    if isinstance(eigenstate, Counts):
        ax.bar([key for key in eigenstate], [eigenstate[key] for key in eigenstate])
        
    for tick in ax.get_xticklabels():
        tick.set_rotation(90)
        
    return fig, ax

## Specify Graph

In [ ]:
edges: List[Tuple[int, int, float]] = [
    (0, 1, 1.0),
    (0, 2, 1.0),
    (1, 2, 1.0),
    (1, 3, 1.0),
    (2, 3, 1.0),
]
        
graph = networkx.Graph()
graph.add_weighted_edges_from(edges)
N = graph.number_of_nodes()
positions = networkx.spring_layout(graph)
    
""" random graph
mat = random_graph(4, edge_prob=1.0, negative_weight=False)
edges = []
for i in range(4):
    for j in range(i):
        edges.append((i, j, mat[i,j]))

        
graph = networkx.Graph()
graph.add_weighted_edges_from(edges)
N = graph.number_of_nodes()
positions = networkx.planar_layout(graph)
"""

In [ ]:
draw_graph(["C0" for node in graph.nodes()])

## Setup Problem for Qiskit

First we obtain the weight matrix of the graph:

In [ ]:
weight_matrix = networkx.convert_matrix.to_numpy_array(graph)
weight_matrix

Qiskit provides a handy routine to obtain the Ising Hamiltonian associated with the Maximum-Cut problem. It returns a weighted Ising operator and an energy offset from the constant term.

In [ ]:
hamiltonian, offset = max_cut.get_operator(weight_matrix)
print("Hamiltonian:")
print("------------")
print(hamiltonian)
print("energy offset:", offset)
print(hamiltonian.print_details())

## Initialize Random Seeds

In [ ]:
aqua.aqua_globals.random_seed = numpy.random.default_rng(498615)
seed = 198687

## Initialize Noise Model

In [ ]:
provider = IBMQ.load_account()
backend = provider.get_backend("ibmq_vigo")
noise_model = NoiseModel.from_backend(backend)
coupling_map = backend.configuration().coupling_map
basis_gates = noise_model.basis_gates

# Solve the Problem

## Brute-Force

In [ ]:
best_profit = 0.0

for combination in range(2 ** N):
    # generate solution candidates (lists of 0's and 1's):
    # 1. bin() converts to binary string
    # 2. [:2] removes the '0b' prefix
    # 3. .zfill(N) prepends 0s until a length of N has been achieved
    binary = generate_binary_list(combination)

    # evaluate the cost function
    profit = 0.0
    for i in range(N):
        for j in range(N):
            profit += weight_matrix[i, j] * binary[i] * (1 - binary[j])

    # check if we found a better solution
    if profit > best_profit:
        best_profit = profit
        solution = binary

    # print info about current combination
    print(
        "combination {}: binary = {}, profit = {}".format(
            combination, str(binary), profit
        )
    )

print()
print("optimal solution: binary = {}, profit = {}".format(str(solution), best_profit))

draw_graph(["C1" if solution[i] else "C0" for i in range(N)])

## Diagonalize Ising Hamiltonian

In [ ]:
result_diag = NumPyMinimumEigensolver(hamiltonian).run()
energy_diag = result_diag.eigenvalue.real + offset

state = sample_most_likely(result_diag.eigenstate)
print("ground state energy:", energy_diag)
print("most likely binary string:", state)
draw_graph(["C1" if state[i] else "C0" for i in range(N)])

## QAOA

In [ ]:
QAOAResult = Tuple[QAOA, MinimumEigensolverResult, numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray]

def run_qaoa(p: int, quantum_instance: QuantumInstance, optimizer) -> QAOAResult:
    evaluations: List[float] = []
    parameters: List[numpy.ndarray] = []
    means: List[float] = []
    stddevs: List[float] = []

    def callback(evals: int, params: numpy.ndarray, mean: float, stddev: float):
        evaluations.append(evals)
        parameters.append(params)
        means.append(mean)
        stddevs.append(stddev)
        
    algorithm = QAOA(hamiltonian, optimizer, quantum_instance=quantum_instance, p=p, callback=callback, initial_point=numpy.random.random(2*p)*2*numpy.pi)
    
    result = algorithm, algorithm.compute_minimum_eigenvalue(), numpy.array(evaluations), numpy.array(parameters), numpy.array(means), numpy.array(stddevs)
    
    return result

### Ideal

In [ ]:
result_qaoa_ideal = run_qaoa(8, QuantumInstance(StatevectorSimulator(), seed_simulator=seed, seed_transpiler=seed), COBYLA())

In [ ]:
plot_convergence(result_qaoa_ideal[4]+offset, result_qaoa_ideal[5])

In [ ]:
plot_parameter_convergence(result_qaoa_ideal[3])

In [ ]:
result_qaoa_ideal[1].eigenstate
#fig,ax = plt.subplots(1,1)
#eigenstate = result[1].eigenstate / numpy.sum(numpy.abs(result[1].eigenstate))
#ax.plot(numpy.abs(eigenstate), marker=".", linestyle=None)

### Noisy

In [ ]:
result_qaoa_noisy = run_qaoa(8, QuantumInstance(
    QasmSimulator(method="statevector"),
    seed_simulator=seed,
    seed_transpiler=seed,
    coupling_map=coupling_map,
    noise_model=noise_model,
    measurement_error_mitigation_cls=CompleteMeasFitter
), COBYLA())

In [ ]:
plot_convergence(result_qaoa_noisy[4]+offset, result_qaoa_noisy[5])

In [ ]:
plot_parameter_convergence(result_qaoa_noisy[3])

In [ ]:
plot_eigenstate(result_qaoa_noisy[1].eigenstate)

## VQE with Ad-Hoc State

In [ ]:
circuit = TwoLocal(hamiltonian.num_qubits, "ry", "cz", reps=5, entanglement="linear")
circuit.draw(filename="circuit.svg")
circuit

In [ ]:
VQEResult = Tuple[VQE, MinimumEigensolverResult, numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray]

def run_vqe(quantum_instance: QuantumInstance, optimizer) -> VQEResult:
    evaluations: List[float] = []
    parameters: List[numpy.ndarray] = []
    means: List[float] = []
    stddevs: List[float] = []

    def callback(evals: int, params: numpy.ndarray, mean: float, stddev: float):
        evaluations.append(evals)
        parameters.append(params)
        means.append(mean)
        stddevs.append(stddev)
        
    algorithm = VQE(hamiltonian, circuit, optimizer, quantum_instance=quantum_instance, callback=callback)
    
    result = algorithm, algorithm.compute_minimum_eigenvalue(), numpy.array(evaluations), numpy.array(parameters), numpy.array(means), numpy.array(stddevs)
    
    return result

### Ideal

In [ ]:
result_vqe_ideal = run_vqe(QuantumInstance(StatevectorSimulator(), seed_simulator=seed, seed_transpiler=seed), COBYLA())

In [ ]:
plot_convergence(result_vqe_ideal[4]+offset, result_vqe_ideal[5])

In [ ]:
plot_parameter_convergence(result_vqe_ideal[3])

### Noisy

In [ ]:
result_vqe_noisy = run_vqe(QuantumInstance(
    QasmSimulator(method="statevector"),
    seed_simulator=seed,
    seed_transpiler=seed,
    coupling_map=coupling_map,
    noise_model=noise_model,
    measurement_error_mitigation_cls=CompleteMeasFitter
), COBYLA())

In [ ]:
plot_convergence(result_vqe_noisy[4]+offset, result_vqe_noisy[5])

In [ ]:
plot_parameter_convergence(result_vqe_noisy[3])

In [ ]:
plot_eigenstate(result_vqe_noisy[1].eigenstate)

In [ ]:
draw_graph(["C1" if [0,1,1,0][i] else "C0" for i in range(N)])

In [ ]:
draw_graph(["C1" if [1,0,0,1][i] else "C0" for i in range(N)])

In [ ]:
draw_graph(["C1" if [1,0,0,0][i] else "C0" for i in range(N)])